# 論拠データを査読者に公開する
査読者が論拠データを閲覧できる環境を用意するタスクです。<br>
Gakunin RDMに論拠データを公開する環境(査読用PJ)を作成し、査読者に公開したいデータのみを格納してください。

## 査読対象のデータを格納する
論拠データ公開用の別プロジェクト（査読用PJ）をこのプロジェクトのコンポーネントとして作成し、査読者の公開するデータを格納して下さい。<br>
査読用PJの作成はGakuNin RDM上で行います。<br>
以下の図の指示に従って査読用PJを作成した後、データを格納してください。<br>
※この公開方法は投稿先の規定や査読方法によっては利用できない可能性があるので注意してください。<br>
（例：ダブルブラインド査読の場合はプロジェクト作成者がわかってしまうので利用できません。）<br>
![コンポーネントの作成](./images/RF005001_create_component.png)


In [ ]:
# 公開環境を作成する
import os

from IPython.display import display
from IPython.core.display import Javascript
import panel as pn

from library.task_director import TaskDirector
from library.utils.config import connect as con_config, message as msg_config
from library.utils.html.button import create_button
from library.utils.input import get_project_id
from library.utils.storage_provider import grdm


notebook_name = 'publish_argument_data_to_reviewers.ipynb'

def access_grdm_main(working_path: str):
    """GakuNin RDMのメイン画面を表示するメソッドです。

    Args:
        working_path (str): 実行ノートブックパス

    """
    task_director = TaskDirector(working_path, notebook_name)
    task_director.doing_task()

    grdm_url = con_config.get('GRDM', 'BASE_URL')
    access_grdm = grdm.Grdm()

    project_id = get_project_id()
    button_width = 500
    obj = create_button(
        url=access_grdm.build_main_menu_url(grdm_url, project_id),
        msg=msg_config.get('publish_argument_data_to_reviewers', 'go_grdm'),
        target='_blank',
        button_width=f'{button_width}px'
    )
    pn.extension()
    display(pn.pane.HTML(obj, width=button_width))
    display(Javascript('IPython.notebook.save_checkpoint();'))
    task_director.done_task()

access_grdm_main(os.path.abspath('__file__'))

### 共同研究者を追加する
共同研究者が査読用PJを操作する場合、作成した査読用PJに共同研究者を追加する必要があります。<br>
査読用のPJを選択し、現在のメンバーを確認してください。<br>
共同研究者のメンバーへの追加はGakuNin RDMの「メンバー」メニューから行えます。

In [ ]:
# 共同研究者を追加する
import os
import traceback

from IPython.core.display import Javascript
from IPython.display import display, clear_output
from requests.exceptions import RequestException
import pandas as pd
import panel as pn

from library.task_director import TaskDirector
from library.utils.config import connect as con_config, message as msg_config
from library.utils.error import (UnauthorizedError, ProjectNotExist,
                                UnusableVault, RepoPermissionError)
from library.utils.html.button import create_button
from library.utils.input import get_grdm_connection_parameters
from library.utils.storage_provider import grdm
from library.utils.widgets import MessageBox


notebook_name = 'publish_argument_data_to_reviewers.ipynb'

class CollaboratorManager(TaskDirector):
    """共同研究者の情報を表示するクラスです。

    Attributes:
        instance:
            _form_box (pn.WidgetBox) : フォームを格納する。
            _msg_output (MessageBox) : ユーザーに提示するメッセージを格納する。
            token(str):パーソナルアクセストークン
            project_id(str):プロジェクトID
            grdm_url(str):GRDMのURL
            grdm(Grdm):grdmファイルのGrdmクラス

    """
    def __init__(self, working_path: str) -> None:
        """CollaboratorManager コンストラクタのメソッドです。

        Args:
            working_path(str):実行Notebookファイルパス
        """
        super().__init__(working_path, notebook_name)

        # フォームボックス
        pn.extension('tabulator')
        self.form_section = pn.WidgetBox()
        # メッセージ用ボックス
        self._msg_output = MessageBox()
        self._msg_output.width = 900
        # コンポーネント用ボックス
        self.component_form_box = pn.WidgetBox()
        self.component_form_box.width = 900

        self.grdm_url = con_config.get('GRDM', 'BASE_URL')
        self.grdm = grdm.Grdm()

    def get_grdm_params(self) -> tuple[str, str]:
        """GRDMのトークンとプロジェクトIDを取得するメソッドです。

        Returns:
            str:GRDMのトークンの値を返す。
            str:プロジェクトIDの値を返す。
        """
        token = ""
        project_id = ""
        try:
            token, project_id = get_grdm_connection_parameters(self.grdm_url)
        except UnusableVault as e:
            message = msg_config.get('form', 'no_vault')
            self._msg_output.update_error(message)
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except RepoPermissionError:
            message = msg_config.get('form', 'insufficient_permission')
            self._msg_output.update_error(message)
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except ProjectNotExist as e:
            self._msg_output.update_error(str(e))
            self.log.error(traceback.format_exc())
        except RequestException as e:
            message = msg_config.get('DEFAULT', 'connection_error')
            self._msg_output.update_error(f'{message}\n{str(e)}')
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self._msg_output.update_error(message)
            self.log.error(message)
        return token, project_id

    @TaskDirector.task_cell("2")
    def generate_collaborator_section(self):
        """取得したデータから共同研究者を表示するメソッドです。"""

        # フォーム定義
        try:
            self.token, self.project_id = self.get_grdm_params()
            clear_output()
            if not self._msg_output.has_message():
                self.form_section.clear()
                # 作業を行っているプロジェクトの共同研究者を表示
                self.generate_working_section()

                # コンポーネントを検索
                self.components = self.grdm.get_component_list(self.grdm_url, self.token, self.project_id)

                # コンポーネントが存在しない場合はメッセージを表示する
                if not self.components:
                    self._msg_output.update_warning(msg_config.get('publish_argument_data_to_reviewers', 'no_component'))
                # コンポーネントのセレクトボックスの設定
                else:
                    self.components[msg_config.get('publish_argument_data_to_reviewers', 'default_component_select')] = 'default'  # Default
                    self.component_select = pn.widgets.Select(name=msg_config.get('publish_argument_data_to_reviewers', 'select_title'), options=self.components, value="default")
                    self.component_select.param.watch(self.on_select_change, 'value')

        except UnauthorizedError:
            message = msg_config.get('form', 'token_unauthorized')
            self._msg_output.update_warning(message)
            self.log.warning(f'{message}\n{traceback.format_exc()}')
        except ProjectNotExist:
            message = msg_config.get('form', 'project_id_not_exist').format(self.project_id)
            self._msg_output.update_error(message)
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except RequestException as e:
            message = msg_config.get('DEFAULT', 'connection_error')
            self._msg_output.update_error(f'{message}\n{str(e)}')
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self._msg_output.update_error(message)
            self.log.error(message)

        # フォーム表示

        self.form_section.append(self.component_select)
        self.form_section.append(self._msg_output)
        display(self.form_section)
        display(Javascript('IPython.notebook.save_checkpoint();'))

    def generate_working_section(self):
        """現在作業を行っているプロジェクトの共同研究者を表示するメソッドです。"""

        # 表示するアイテムの作成
        contents = self.grdm.get_collaborator_list(self.grdm_url, self.token, self.project_id)
        table = self.display_table(contents)
        button = self.generate_button(self.project_id)

        working_form_box = pn.WidgetBox()
        working_form_box.width = 900
        # フォーム定義
        title = msg_config.get('manage_collaborators', 'table_title')
        working_form_box.append(f'# {title}')
        working_form_box.append(table)
        working_form_box.append(button)

        self.form_section.append(working_form_box)

    def on_select_change(self, event):
        """セレクトボックスでコンポーネントが選択されたときに共同研究者を表示するメソッドです。"""

        selected_value = self.component_select.value

        if selected_value != 'default':
            contents = self.grdm.get_collaborator_list(self.grdm_url, self.token, selected_value)
            table = self.display_table(contents)
            button = self.generate_button(selected_value)

            title = msg_config.get('manage_collaborators', 'table_title')
            for key, value in self.components.items():
                if value == selected_value:
                    selected_key = key
                    break

            self.component_form_box.clear()
            self.component_form_box.append(f'# {title}({selected_key})')
            self.component_form_box.append(table)
            self.component_form_box.append(button)

            self.form_section.append(self.component_form_box)

    def generate_button(self, project_id: str) -> pn.pane.HTML:
        """メンバー一覧のページを表示するボタンを作成するメソッドです。

        Args:
            project_id(str):プロジェクトID

        Returns:
            pn.pane.HTML:ボタンを表示させる。

        """
        button_width = 500
        obj = create_button(
            url=self.grdm.build_collaborator_url(self.grdm_url, project_id),
            msg=msg_config.get('manage_collaborators', 'go_grdm'),
            target='_blank',
            button_width=f'{button_width}px'
        )
        return pn.pane.HTML(obj, width=button_width)

    def display_table(self, contents: dict) -> pn.widgets.Tabulator:
        """共同研究者の持つ情報を出力するメソッドです。

        共同研究者の名前や管理者、書き込み、読み取りの内、どの権限を持っているかなどを出力する。

        Args:
            contents(dict):共同管理者の情報

        Returns:
            pn.widgets.Tabulator:共同研究者の情報を表示する表

        """
        display_permission = {
                "admin": msg_config.get('manage_collaborators', 'admin'),
                "write": msg_config.get('manage_collaborators', 'write'),
                "read": msg_config.get('manage_collaborators', 'read')
            }

        name_list = []
        permission_list = []
        for full_name, permission in contents.items():
            name_list.append(full_name)
            permission_list.append(display_permission[permission])

        df = pd.DataFrame({
            msg_config.get('manage_collaborators', 'name_title'): name_list,
            msg_config.get('manage_collaborators', 'permission_title'): permission_list
        })
        return pn.widgets.Tabulator(
            df,
            show_index=False,
            header_align='center',
            text_align='center',
            configuration={'columnDefaults': {'headerSort': False}},
            editors={
                msg_config.get('manage_collaborators', 'name_title'): None,
                msg_config.get('manage_collaborators', 'permission_title'): None
            }
        )

CollaboratorManager(working_path=os.path.abspath('__file__')).generate_collaborator_section()

## 査読者が再現可能な環境を作成する
査読者に公開した論拠データが再現可能な状態であることを確認してください。<br>
※再現性を確認する機能を現在開発中のため、再現性の確認は手動で行ってください。

## 査読者の権限を設定する
投稿先（査読者）用のGRDMアカウントを査読用PJのメンバーに追加してください。<br>
メンバーへの追加はGakuNin RDMの「メンバー」メニューから行えます。<br>
※査読者のアカウントをメンバーに追加する際は権限を読み取り専用に設定してください。

In [ ]:
# 査読用PJの「メンバー」を表示する
import os
import traceback

from IPython.core.display import Javascript
from IPython.display import display, clear_output
from requests.exceptions import RequestException
import pandas as pd
import panel as pn

from library.task_director import TaskDirector
from library.utils.config import connect as con_config, message as msg_config
from library.utils.error import (UnauthorizedError, ProjectNotExist,
                                UnusableVault, RepoPermissionError)
from library.utils.html.button import create_button
from library.utils.input import get_grdm_connection_parameters
from library.utils.storage_provider import grdm
from library.utils.widgets import MessageBox


notebook_name = 'publish_argument_data_to_reviewers.ipynb'

class CollaboratorManager(TaskDirector):
    """共同研究者の情報を表示するクラスです。

    Attributes:
        instance:
            _form_box (pn.WidgetBox) : フォームを格納する。
            _msg_output (MessageBox) : ユーザーに提示するメッセージを格納する。
            token(str):パーソナルアクセストークン
            project_id(str):プロジェクトID
            grdm_url(str):GRDMのURL
            grdm(Grdm):grdmファイルのGrdmクラス

    """
    def __init__(self, working_path: str) -> None:
        """CollaboratorManager コンストラクタのメソッドです。

        Args:
            working_path(str):実行Notebookファイルパス

        """
        super().__init__(working_path, notebook_name)

        # フォームボックス
        pn.extension('tabulator')
        self.form_section = pn.WidgetBox()
        # メッセージ用ボックス
        self._msg_output = MessageBox()
        self._msg_output.width = 900

        # コンポーネント用ボックス
        self.component_form_box = pn.WidgetBox()
        self.component_form_box.width = 900

        self.grdm_url = con_config.get('GRDM', 'BASE_URL')
        self.grdm = grdm.Grdm()

    def get_grdm_params(self) -> tuple[str, str]:
        """GRDMのトークンとプロジェクトIDを取得するメソッドです。

        Returns:
            str:GRDMのトークンの値を返す。
            str:プロジェクトIDの値を返す。

        """
        token = ""
        project_id = ""
        try:
            token, project_id = get_grdm_connection_parameters(self.grdm_url)
        except UnusableVault as e:
            message = msg_config.get('form', 'no_vault')
            self._msg_output.update_error(message)
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except RepoPermissionError:
            message = msg_config.get('form', 'insufficient_permission')
            self._msg_output.update_error(message)
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except ProjectNotExist as e:
            self._msg_output.update_error(str(e))
            self.log.error(traceback.format_exc())
        except RequestException as e:
            message = msg_config.get('DEFAULT', 'connection_error')
            self._msg_output.update_error(f'{message}\n{str(e)}')
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self._msg_output.update_error(message)
            self.log.error(message)
        return token, project_id

    @TaskDirector.task_cell("3")
    def generate_collaborator_section(self):
        """取得したデータから共同研究者を表示するメソッドです。"""

        # フォーム定義
        try:
            self.token, self.project_id = self.get_grdm_params()
            clear_output()
            if not self._msg_output.has_message():
                self._form_box.clear()
                # コンポーネントを検索
                self.components = self.grdm.get_component_list(self.grdm_url, self.token, self.project_id)

                # コンポーネントが存在しない場合はメッセージを表示する
                if not self.components:
                    self._msg_output.update_warning(msg_config.get('publish_argument_data_to_reviewers', 'no_component'))
                # コンポーネントのセレクトボックスの設定
                else:
                    self.components[msg_config.get('publish_argument_data_to_reviewers', 'default_component_select')] = 'default'  # Default
                    self.component_select = pn.widgets.Select(name=msg_config.get('publish_argument_data_to_reviewers', 'select_title'), options=self.components, value="default")
                    self.component_select.param.watch(self.on_select_change, 'value')

        except UnauthorizedError:
            message = msg_config.get('form', 'token_unauthorized')
            self._msg_output.update_warning(message)
            self.log.warning(f'{message}\n{traceback.format_exc()}')
        except ProjectNotExist:
            message = msg_config.get('form', 'project_id_not_exist').format(self.project_id)
            self._msg_output.update_error(message)
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except RequestException as e:
            message = msg_config.get('DEFAULT', 'connection_error')
            self._msg_output.update_error(f'{message}\n{str(e)}')
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self._msg_output.update_error(message)
            self.log.error(message)

        # フォーム表示
        self.form_section.append(self.component_select)
        self.form_section.append(self._msg_output)
        display(self.form_section)
        display(Javascript('IPython.notebook.save_checkpoint();'))

    def on_select_change(self, event):
        """セレクトボックスでコンポーネントが選択されたときに共同研究者を表示するメソッドです。"""

        selected_value = self.component_select.value

        if selected_value != 'default':
            contents = self.grdm.get_collaborator_list(self.grdm_url, self.token, selected_value)
            table = self.display_table(contents)
            button = self.generate_button(selected_value)

            title = msg_config.get('manage_collaborators', 'table_title')
            for key, value in self.components.items():
                if value == selected_value:
                    selected_key = key
                    break

            self.component_form_box.clear()
            self.component_form_box.append(f'# {title}({selected_key})')
            self.component_form_box.append(table)
            self.component_form_box.append(button)

            self.form_section.append(self.component_form_box)

    def generate_button(self, project_id: str) -> pn.pane.HTML:
        """メンバー一覧のページを表示するボタンを作成するメソッドです。

        Args:
            project_id(str):プロジェクトID

        Returns:
            pn.pane.HTML:ボタンを表示させる。

        """
        button_width = 500
        obj = create_button(
            url=self.grdm.build_collaborator_url(self.grdm_url, project_id),
            msg=msg_config.get('manage_collaborators', 'go_grdm'),
            target='_blank',
            button_width=f'{button_width}px'
        )
        return pn.pane.HTML(obj, width=button_width)

    def display_table(self, contents: dict) -> pn.widgets.Tabulator:
        """共同研究者の持つ情報を出力するメソッドです。

        共同研究者の名前や管理者、書き込み、読み取りの内、どの権限を持っているかなどを出力する。

        Args:
            contents(dict):共同管理者の情報

        Returns:
            pn.widgets.Tabulator:共同研究者の情報を表示する表

        """
        display_permission = {
                "admin": msg_config.get('manage_collaborators', 'admin'),
                "write": msg_config.get('manage_collaborators', 'write'),
                "read": msg_config.get('manage_collaborators', 'read')
            }

        name_list = []
        permission_list = []
        for full_name, permission in contents.items():
            name_list.append(full_name)
            permission_list.append(display_permission[permission])

        df = pd.DataFrame({
            msg_config.get('manage_collaborators', 'name_title'): name_list,
            msg_config.get('manage_collaborators', 'permission_title'): permission_list
        })
        return pn.widgets.Tabulator(
            df,
            show_index=False,
            header_align='center',
            text_align='center',
            configuration={'columnDefaults': {'headerSort': False}},
            editors={
                msg_config.get('manage_collaborators', 'name_title'): None,
                msg_config.get('manage_collaborators', 'permission_title'): None
            }
        )

CollaboratorManager(working_path=os.path.abspath('__file__')).generate_collaborator_section()

## サブフローメニューを表示する

In [ ]:
# サブフローメニューを表示する
import os
from library.task_director import TaskDirector

script_file_name = "publish_argument_data_to_reviewers"
notebook_name = script_file_name+'.ipynb'
TaskDirector(os.path.abspath('__file__'), notebook_name).return_subflow_menu()